In [11]:
import pymysql

conn = pymysql.connect(
    host='20.39.192.91',
    port=3306,
    user='SE_Tony',
    password='Qkrtnqja!1234',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor  # 여기 추가
)

cursor = conn.cursor()
cursor.execute("SELECT VERSION();")
print("MySQL Version:", cursor.fetchone())

try:
    with conn.cursor() as cursor:
        # 2. 테이블 전체 조회
        cursor.execute("SELECT title, organization, department FROM youth_job_db.job_postings")  # ← 실제 테이블 이름으로 수정
        rows = cursor.fetchall()
        print(f"총 {len(rows)}개의 레코드 가져옴.")
        for row in rows[:3]:  # 처음 3개만 출력해봄
            print(row)
finally:
    conn.close()

MySQL Version: {'VERSION()': '8.0.42-0ubuntu0.24.04.2'}
총 37526개의 레코드 가져옴.
{'title': '국립암센터 부속병원 당직전담의사 초빙공고', 'organization': '국립암센터', 'department': ''}
{'title': '채용형 인턴 채용공고(주택관리공단 강원지사)', 'organization': '주택관리공단(주)', 'department': ''}
{'title': '한국여성정책연구원 2025년 제5차 직원 블라인드 채용 공고 ', 'organization': '한국여성정책연구원', 'department': ''}


In [7]:
print(rows[0])

{'title': '국립암센터 부속병원 당직전담의사 초빙공고', 'organization': '국립암센터'}


In [8]:
import os
from langchain.schema import Document
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI



In [17]:
page_content = f"""
기관명: {row['organization']}
채용 직무: {row['title']}
부서: {row['department'] or '부서 정보 없음'}
"""

documents = [
    Document(
        page_content=f"""
기관명: {row['organization']}
채용 직무: {row['title']}
부서: {row['department'] or '부서 정보 없음'}
""",
    )
    for row in rows
]
print(f"Loaded {len(documents)} documents from database.")
print(f"First document content: {documents[0].page_content[:200]}...")  # Print first 200 characters of the first document
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
print(f"text_splitter: {text_splitter} \n")  # Print the text splitter configuration for debugging
docs = text_splitter.split_documents(documents)
print(f"Split into {len(docs)} chunks. \n")
#print(f"docs: {docs} \n") 
print(f"First chunk content: {docs[0]}...")  # Print first 200 characters of the first chunk



Loaded 37526 documents from database.
First document content: 
기관명: 국립암센터
채용 직무: 국립암센터 부속병원 당직전담의사 초빙공고
부서: 부서 정보 없음
...
text_splitter: <langchain_text_splitters.character.CharacterTextSplitter object at 0x11a546650> 

Split into 37526 chunks. 

First chunk content: page_content='기관명: 국립암센터
채용 직무: 국립암센터 부속병원 당직전담의사 초빙공고
부서: 부서 정보 없음'...


In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_key="DaH9GW5nK15zX9b1541B4H7lt2l9YeXLm6xzWzrdhjlZw322zTINJQQJ99BGACYeBjFXJ3w3AAABACOGbelF",
    openai_api_version="2023-05-15", 
    azure_endpoint="https://rounders-openai.openai.azure.com",  
    chunk_size=1000,
)
print(f"Embeddings initialized: {embeddings}")
vectorstore = FAISS.from_documents(docs, embeddings)

Embeddings initialized: client=<openai.resources.embeddings.Embeddings object at 0x116f24700> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x1390ff5b0> model='text-embedding-ada-002' deployment='text-embedding-ada-002' openai_api_version='2023-05-15' openai_api_base=None openai_api_type='azure' openai_proxy='' embedding_ctx_length=8191 openai_api_key='DaH9GW5nK15zX9b1541B4H7lt2l9YeXLm6xzWzrdhjlZw322zTINJQQJ99BGACYeBjFXJ3w3AAABACOGbelF' openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None azure_endpoint='https://rounders-openai.openai.azure.com' azure_ad_token=None azure_ad_token_provider=None azure_ad_async_token_provider=None validate_base_url=True


KeyboardInterrupt: 

In [ ]:
print(vectorstore)
for i, (doc_id, doc) in enumerate(vectorstore.docstore._dict.items()):
    print(f"[Document {i+1}]")
    print(f"내용: {doc.page_content[:300]}...\n")  # 처음 300자만 출력

print(f"벡터 개수: {vectorstore.index.ntotal}")


In [ ]:
llm = AzureChatOpenAI(
    deployment_name="o4-mini",  
    openai_api_key="DaH9GW5nK15zX9b1541B4H7lt2l9YeXLm6xzWzrdhjlZw322zTINJQQJ99BGACYeBjFXJ3w3AAABACOGbelF",
    api_version="2024-12-01-preview",
    azure_endpoint="https://rounders-openai.openai.azure.com/",
    temperature=1 ,
)
print("LLM initialized successfully.", llm)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",  # or "map_reduce", "refine" if 문서 많을 때
)




In [ ]:
query = "내가 지금까지 한 활동들을 현대 오토에버에 맞는 자소서로 작성해줘"
answer = qa.run(query)
print("📌 답변:", answer)